In [83]:
import pandas as pd
import numpy as np
import plotly.express as px
import datetime

In [84]:
# Laden der Parquet Datei in einen Dataframe
df = pd.read_parquet('escooter_history.parquet')

In [85]:
# Dataframe 8 Jahre zurückverschieben um Holiday und workindday konsistenz zu gewährleisten
df["datetime"] = df["datetime"] - datetime.timedelta(2922)

# Erstellung des stündlichen DF

In [86]:
# gruppieren nach stunden summe von registrierten kunden und zählen von index
df_count_hourly = df[['datetime','registered_customer']].resample('1H',on='datetime').agg({'datetime':'count','registered_customer':'sum'}).rename(columns={'datetime':'Bookings'})
df_count_hourly.head()
df_count_hourly['unregistered_customer'] = df_count_hourly['Bookings'] - df_count_hourly['registered_customer']

In [87]:
# gruppieren nach stunden
df_mean_hourly = df.resample('1H',on='datetime').mean()
df_mean_hourly.drop(['registered_customer'],axis=1,inplace=True)

In [88]:
df_hourly = pd.merge(df_count_hourly,df_mean_hourly,on='datetime').reset_index()
df_hourly.head()

,datetime,Bookings,registered_customer,unregistered_customer,holiday,workingday,temp,atemp,humidity,windspeed
0,2011-01-01 00:00:00,16,13,3,0.0,0.0,9.84,14.395,81.0,0.0
1,2011-01-01 01:00:00,40,32,8,0.0,0.0,9.02,13.635,80.0,0.0
2,2011-01-01 02:00:00,32,27,5,0.0,0.0,9.02,13.635,80.0,0.0
3,2011-01-01 03:00:00,13,10,3,0.0,0.0,9.84,14.395,75.0,0.0
4,2011-01-01 04:00:00,1,1,0,0.0,0.0,9.84,14.395,75.0,0.0


In [89]:
#### Hinzufügen von ergänzenden Werten
df_hourly['day_of_week'] = df_hourly['datetime'].dt.dayofweek
df_hourly['weekday'] = df_hourly['datetime'].dt.day_name()
season_dict = { 1 : "Winter", 2 : "Spring", 3 : "Summer", 4 :"Autumn"}
df_hourly["season"] = df_hourly["datetime"].dt.month % 12 // 3 + 1
df_hourly["season"] =  df_hourly["season"].map(season_dict)
df_hourly['week_of_year'] =  df_hourly['datetime'].dt.isocalendar().week
df_hourly['hour'] = df_hourly['datetime'].dt.hour
df_hourly['day'] = df_hourly['datetime'].dt.day
df_hourly['month'] = df_hourly['datetime'].dt.month
df_hourly['year'] =  df_hourly['datetime'].dt.year
df_hourly['date'] = df_hourly['datetime'].dt.date

In [90]:
# löschen von Stunden an denen keine Buchung vor kam
df_hourly = df_hourly[~(df_hourly['Bookings']==0)]

# Erstellung des täglichen DF

In [91]:
df_day = df.set_index('datetime')
df_day.index = pd.to_datetime(df_day.index)
df_day_mean = df_day.resample('d').mean()
df_day_mean = df_day_mean.drop('registered_customer', axis = 1)
df_day_bookings = df_day.resample('d')[['temp', 'registered_customer']].agg({'temp' : 'count', 'registered_customer' : 'sum'})
df_day_bookings.columns = ['Bookings', 'registered_customer']
df_day = pd.merge(df_day_mean.reset_index(), df_day_bookings.reset_index(), on = 'datetime')
# df_day['weather'] = df_day['weather'].astype('int')
df_day['unregistered_customer'] = df_day.Bookings - df_day.registered_customer
df_day["holiday"] = df_day["holiday"].astype(int)
df_day["workingday"] = df_day["workingday"].astype(int)
df_day

,datetime,holiday,workingday,temp,atemp,humidity,windspeed,Bookings,registered_customer,unregistered_customer
0,2011-01-01,0,0,16.489909,20.405660,79.678173,16.226271,985,654,331
1,2011-01-02,0,0,14.415006,16.837996,67.141074,15.875136,801,670,131
2,2011-01-03,0,1,8.440104,10.139789,40.287620,14.724723,1349,1229,120
3,2011-01-04,0,1,9.051498,11.263326,56.411012,12.351703,1562,1454,108
4,2011-01-05,0,1,9.649350,11.463716,38.230000,14.113181,1600,1518,82
...,...,...,...,...,...,...,...,...,...,...
726,2012-12-27,0,1,10.475495,11.515609,59.472229,22.242182,4231,4024,207
727,2012-12-28,0,1,10.448490,12.771144,58.453446,10.219380,4629,4381,248
728,2012-12-29,0,0,10.684172,12.120000,77.426304,7.881757,3087,2777,310
729,2012-12-30,0,0,10.983624,12.108261,44.246362,24.581467,2886,2440,446


In [92]:
# Erstellung eines dummy DF für die Wetter-Attribute
df_dummy = pd.get_dummies(df[['datetime', 'weather']])
df_dummy['Datum'] = df_dummy.datetime.dt.date
df_dummy = df_dummy.groupby(['Datum']).sum()
df_dummy = df_dummy.reset_index()
df_dummy['Datum'] = pd.to_datetime(df_dummy.Datum)

def get_max_dummy(val):
    st = val.idxmax()
    val[st] = 1
    li = ['weather_clear, few clouds', 'weather_cloudy, mist' ,'weather_heavy rain or thunderstorm or snow or ice pallets','weather_light snow or rain or thunderstorm']
    li.remove(st)
    for el in li:
        val[el] = 0
    return val

df_dummyt = df_dummy.drop('Datum', axis = 1).apply(get_max_dummy, axis=1)
df_dummy = pd.merge(df_dummy[['Datum']].reset_index(), df_dummyt.reset_index(), on='index')
df_dummy = df_dummy.drop('index', axis = 1)
df_dummy = df_dummy.set_index('Datum')
df_dummy = df_dummy.astype(int)

In [93]:
# Hinzufügen der Dummy Wetter Attribute
df_day = pd.merge(df_day, df_dummy.reset_index(), left_on='datetime', right_on='Datum')

In [94]:
# Zeitwerte werden hinzugefügt
df_day['day_of_week'] = df_day['datetime'].dt.dayofweek
df_day['season'] =  df_day['datetime'].dt.month%12 // 3 + 1
df_day['week_of_year'] =  df_day['datetime'].dt.isocalendar().week
df_day['day'] = df_day['datetime'].dt.day
df_day['month'] = df_day['datetime'].dt.month
df_day['year'] =  df_day['datetime'].dt.year
df_day

,datetime,holiday,workingday,temp,atemp,humidity,windspeed,Bookings,registered_customer,unregistered_customer,...,"weather_cloudy, mist",weather_heacy rain or thunderstorm or snow or ice pallets,weather_light snow or rain or thunderstorm,weather_heavy rain or thunderstorm or snow or ice pallets,day_of_week,season,week_of_year,day,month,year
0,2011-01-01,0,0,16.489909,20.405660,79.678173,16.226271,985,654,331,...,1,0,0,0,5,1,52,1,1,2011
1,2011-01-02,0,0,14.415006,16.837996,67.141074,15.875136,801,670,131,...,1,0,0,0,6,1,52,2,1,2011
2,2011-01-03,0,1,8.440104,10.139789,40.287620,14.724723,1349,1229,120,...,0,0,0,0,0,1,1,3,1,2011
3,2011-01-04,0,1,9.051498,11.263326,56.411012,12.351703,1562,1454,108,...,0,0,0,0,1,1,1,4,1,2011
4,2011-01-05,0,1,9.649350,11.463716,38.230000,14.113181,1600,1518,82,...,0,0,0,0,2,1,1,5,1,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,2012-12-27,0,1,10.475495,11.515609,59.472229,22.242182,4231,4024,207,...,0,0,0,0,3,1,52,27,12,2012
727,2012-12-28,0,1,10.448490,12.771144,58.453446,10.219380,4629,4381,248,...,1,0,0,0,4,1,52,28,12,2012
728,2012-12-29,0,0,10.684172,12.120000,77.426304,7.881757,3087,2777,310,...,1,0,0,0,5,1,52,29,12,2012
729,2012-12-30,0,0,10.983624,12.108261,44.246362,24.581467,2886,2440,446,...,0,0,0,0,6,1,52,30,12,2012


# Regression

In [134]:
# mask = df_day.year == 2011
px.scatter(df_day,x = 'atemp', y = 'Bookings', color )

In [125]:
df_day

,datetime,holiday,workingday,temp,atemp,humidity,windspeed,Bookings,registered_customer,unregistered_customer,...,weather_heacy rain or thunderstorm or snow or ice pallets,weather_light snow or rain or thunderstorm,weather_heavy rain or thunderstorm or snow or ice pallets,day_of_week,season,week_of_year,day,month,year,Prediction
0,2011-01-01,0,0,16.489909,20.405660,79.678173,16.226271,985,654,331,...,0,0,0,5,1,52,1,1,2011,1356.484375
1,2011-01-02,0,0,14.415006,16.837996,67.141074,15.875136,801,670,131,...,0,0,0,6,1,52,2,1,2011,846.859375
2,2011-01-03,0,1,8.440104,10.139789,40.287620,14.724723,1349,1229,120,...,0,0,0,0,1,1,3,1,2011,1289.296875
3,2011-01-04,0,1,9.051498,11.263326,56.411012,12.351703,1562,1454,108,...,0,0,0,1,1,1,4,1,2011,1308.359375
4,2011-01-05,0,1,9.649350,11.463716,38.230000,14.113181,1600,1518,82,...,0,0,0,2,1,1,5,1,2011,1314.984375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,2012-12-27,0,1,10.475495,11.515609,59.472229,22.242182,4231,4024,207,...,0,0,0,3,1,52,27,12,2012,4657.281250
727,2012-12-28,0,1,10.448490,12.771144,58.453446,10.219380,4629,4381,248,...,0,0,0,4,1,52,28,12,2012,4764.265625
728,2012-12-29,0,0,10.684172,12.120000,77.426304,7.881757,3087,2777,310,...,0,0,0,5,1,52,29,12,2012,3182.203125
729,2012-12-30,0,0,10.983624,12.108261,44.246362,24.581467,2886,2440,446,...,0,0,0,6,1,52,30,12,2012,3979.343750


In [132]:
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

col = ['atemp', 'year', 'workingday', 
       'weather_clear, few clouds',  'weather_light snow or rain or thunderstorm', 'month'] # 'weather_clear', 'weather_cloudy', 'weather_heavy', 'weather_snow', 
xdf_wm = df_day[col]
# xdf_wm = pd.get_dummies(xdf_wm, columns=['month'])
ydf_wm = df_day[['Bookings']]

# Interaction Term
# poly = PolynomialFeatures(interaction_only=True)
# xdf_wm = poly.fit_transform(xdf_wm)

# Linear Regression
X_train, X_test, y_train, y_test = train_test_split(xdf_wm, ydf_wm, test_size=0.25, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)
print(r2_score(y_train, model.predict(X_train)))
print(r2_score(y_test, model.predict(X_test)))
df_day['Prediction'] = model.predict(xdf_wm)
fig = px.line(df_day, x = 'Datum', y = ['Bookings', 'Prediction'], title = 'Vergleich zwischen Modell und eigentlichen Werten')
fig.update_layout(width = 1200, height = 500, showlegend = False)

0.8008498423385262
0.8348812981680951


In [ ]:
px

In [138]:
df_day['year'] = df_day.year.astype('str')
px.scatter(df_day, x = 'atemp', y = 'Bookings', color = 'year', 
           color_discrete_map= {'2011' : '#3B4B5E', '2012' : '#C19D63'})

In [95]:
df_dummy = pd.get_dummies(df[['datetime', 'weather']], columns=['weather'])
df_dummy = df_dummy.groupby(['datetime']).mean()
df_dummy.index = pd.to_datetime(df_dummy.index)
df_dummy = df_dummy.resample('d').mean()
df_dummy = df_dummy.reset_index()
df_dummy.columns = ['datetime', 'weather_clear', 'weather_cloudy', 'weather_heavy', 'weather_snow']
df_reg = pd.merge(df_day, df_dummy, on = 'datetime')

In [122]:
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

col = ['atemp', 'year', 'workingday', 
       'weather_clear',  'weather_snow', 'month'] # 'weather_clear', 'weather_cloudy', 'weather_heavy', 'weather_snow', 
xdf_wm = df_reg[col]
# xdf_wm = pd.get_dummies(xdf_wm, columns=['month'])
ydf_wm = df_reg[['Bookings']]

# Interaction Term
# poly = PolynomialFeatures(interaction_only=True)
# xdf_wm = poly.fit_transform(xdf_wm)

# Linear Regression
X_train, X_test, y_train, y_test = train_test_split(xdf_wm, ydf_wm, test_size=0.25, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)
print(r2_score(y_train, model.predict(X_train)))
print(r2_score(y_test, model.predict(X_test)))

0.8368692943488796
0.8658734827029758


In [120]:
df_day['Prediction'] = model.predict(xdf_wm)
fig = px.line(df_day, x = 'Datum', y = ['Bookings', 'Prediction'], title = 'Vergleich zwischen Modell und eigentlichen Werten')
fig.update_layout(width = 1200, height = 500, showlegend = False)

# Stündlich

In [98]:
df_dummy = pd.get_dummies(df[['datetime', 'weather']], columns=['weather'])
df_dummy = df_dummy.groupby(['datetime']).mean()
df_dummy.index = pd.to_datetime(df_dummy.index)
df_dummy = df_dummy.resample('h').mean()
df_dummy = df_dummy.reset_index()
df_hourlyy = pd.merge(df_hourly, df_dummy, on = 'datetime')


In [99]:
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures

col = ['atemp', 'humidity','windspeed', 'year','workingday', 
       'month', 'hour', 'weather_clear, few clouds', 'weather_light snow or rain or thunderstorm']
xdf_wm = df_hourlyy[col]
ydf_wm = df_hourlyy[['Bookings']]
xdf_wm = pd.get_dummies(xdf_wm, columns=['month', 'hour'])

# Interaction Term
poly = PolynomialFeatures(interaction_only=True)
xdf_wm = poly.fit_transform(xdf_wm)

# Linear Regression
X_train, X_test, y_train, y_test = train_test_split(xdf_wm, ydf_wm, test_size=0.35, random_state=42)

model = Ridge()
model.fit(X_train, y_train)
print(r2_score(y_train, model.predict(X_train)))
print(r2_score(y_test, model.predict(X_test)))

0.9005240607036361
0.8867733408024304


In [100]:
df_hourlyy['Prediction'] = model.predict(xdf_wm).round()
df_hourlyy.head()
def get_null(val):
    if val < 0:
        return 0
    else:
        return val
df_hourlyy['Prediction'] = df_hourlyy['Prediction'].apply(get_null)
# print(r2_score(df_hourlyy['Bookings'], df_hourlyy['Prediction']))

In [101]:
mask = (df_hourlyy['year'] == 2011) & (df_hourlyy['month'] == 8) & (df_hourlyy['week_of_year'] == 32)
fig = px.line(df_hourlyy[mask].reset_index(), x = 'datetime', y = 'Bookings',
        title = 'Vergleich zwischen den stündlichen Bookings und der Prognose in der Kalenderwoche 32 im Jahr 2011')
fig.add_scatter(x = df_hourlyy[mask].reset_index()['datetime'], y = df_hourlyy[mask]['Prediction'], line = {'dash' : 'dot'}, name='Trend')


In [102]:
df_hourly_series = df_hourly.set_index('datetime')
df_hourly_series = df_hourly_series.resample('m')[['Bookings']].sum()
df_hourly_series

,Bookings
datetime,
2011-01-31,36383
2011-02-28,47149
2011-03-31,64816
2011-04-30,85751
2011-05-31,135840
2011-06-30,142371
2011-07-31,143711
2011-08-31,134420
2011-09-30,118611


# Zeitreihenanalyse

In [103]:
# Resample des DF auf den Monat
df_time = df.set_index('datetime') 
df_time.index = pd.to_datetime(df_time.index) # Sichergehen, dass Index das richtige Format hat. 
df_time = df_time.resample('M').count()[['temp']]
df_time.columns = ['Bookings']
df_time['Vorheriger_Monat'] = df_time['Bookings'].shift()

# Einfügen von Mulitplikativen und Additiven Unterschieden
df_time['Mul'] = df_time['Bookings'] / df_time['Vorheriger_Monat']
df_time['Add'] = df_time['Bookings'] - df_time['Vorheriger_Monat']

# Einfügen des Rolling Average
df_time['Rol_avg'] = df_time['Bookings'].rolling(window = 3).mean()
df_time['Monat_t'] = range(1,25)

In [104]:
# Ermittlung der Regressionsgerade
from sklearn.linear_model import LinearRegression

X = df_time[['Monat_t']]
y = df_time[['Bookings']]

model = LinearRegression(fit_intercept=True) # Verschiebung auf der y-Achse wird zugelassen
model.fit(X,y)
df_time['Prediction'] = model.predict(X)
print('Regressionsfunktion aus Statistik : y = ax + b')
print(f"Koeffizient a = {round(float(model.coef_), 2)}")
print(f"Koeffizient b = {round(float(model.intercept_), 2)}")

Regressionsfunktion aus Statistik : y = ax + b
Koeffizient a = 5760.29
Koeffizient b = 65330.56


In [105]:
# Visualierung des Unterschieds zwischen den Trendgeraden und den monatlichen Werten
df_time['Unterschied_Prognose'] = df_time['Bookings'] - df_time['Prediction']

# Ermittlung der Saisonalen Einflüsse
df_time['Monat'] = pd.DatetimeIndex(df_time.index).month # Monat wird aus dem Index extrahiert. 

# Gruppierung nach dem Monat liefert die durschnittliche Abweichung pro Monat
df_time_seasonal = df_time.groupby(['Monat'], as_index=False)[['Unterschied_Prognose']].mean()
df_time_seasonal.columns = ['Monat', 'Saisonale_Komponente']

# Die durchschnittliche Abweichung wird dem df hinzugefügt
df_time_sc = pd.merge(df_time.reset_index(), df_time_seasonal, on = 'Monat').sort_values('datetime')
df_time_sc = df_time_sc.set_index('datetime')

# durch Addition des Trends mit den monatlichen Werten ergibt sich ein erster Verlauf
df_time_sc['Seasonal_Prediction'] = df_time_sc['Prediction'] + df_time_sc['Saisonale_Komponente']

In [106]:
# Erstellung eines DF welcher das Jahr 2013 enthält
df_2013_22 = pd.DataFrame(pd.date_range(start = '2013-01-01', end='2013-12-31', freq='M'))
df_2013_22.columns = ['datetime']
df_2013_22['Jahr'] = df_2013_22.datetime.dt.year
df_2013_22['Monat'] = df_2013_22.datetime.dt.month
df_2013_22['Monat_t'] = range(25,37)  

# Bildung der Trendgerade
df_2013_22['Prediction'] = model.predict(df_2013_22[['Monat_t']])

# monatliche Abweichungen werden zum df hinzugefügt und addiert
df_2013_22 = pd.merge(df_2013_22, df_time_seasonal, on='Monat').sort_values('datetime')
df_2013_22['Seasonal_Prediction'] = df_2013_22['Prediction'] + df_2013_22['Saisonale_Komponente']
df_2013_22['datetime'] = pd.to_datetime(df_2013_22['datetime'])
df_2013_22 = df_2013_22.set_index('datetime')
df_2013_22

,Jahr,Monat,Monat_t,Prediction,Saisonale_Komponente,Seasonal_Prediction
datetime,,,,,,
2013-01-31,2013,1,25,209337.855072,-40975.603768,168362.251304
2013-02-28,2013,2,26,215098.146812,-37064.895507,178033.251304
2013-03-31,2013,3,27,220858.438551,-5576.187246,215282.251304
2013-04-30,2013,4,28,226618.730290,8038.521014,234657.251304
2013-05-31,2013,5,29,232379.022029,36960.729275,269339.751304
2013-06-30,2013,6,30,238139.313768,39353.937536,277493.251304
2013-07-31,2013,7,31,243899.605507,30052.645797,273952.251304
2013-08-31,2013,8,32,249659.897246,26629.854058,276289.751304
2013-09-30,2013,9,33,255420.188986,15859.562319,271279.751304


In [107]:
# die Prognose für das Jahr 2013 wird an den monatlichen DF angeheftet
df_viz = df_time_sc[['Bookings', 'Seasonal_Prediction', 'Prediction']]
df_viz = pd.concat([df_viz, df_2013_22[['Seasonal_Prediction', 'Prediction']]])
# df_viz

In [108]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.ar_model import AutoReg
import warnings
warnings.filterwarnings("ignore")

model1 = ExponentialSmoothing(df_time_sc['Bookings'],
                             freq = 'M',
                             trend = 'add', 
                             seasonal = 'add',
                             seasonal_periods = 12).fit(smoothing_level = 0.9)

model2 = ExponentialSmoothing(df_time_sc['Bookings'],
                             freq = 'M',
                             trend = 'add', 
                             seasonal = 'add',
                             seasonal_periods = 12).fit(smoothing_level = 0.1)

df_time_sc.index = pd.to_datetime(df_time_sc.index)

model_autoreg = AutoReg(df_time_sc['Bookings'],None,trend='ct', seasonal=True, period = 12) # period in DF information
res = model_autoreg.fit()


df_viz['Exponential_Smoothing0.9'] = model1.predict(start = 0, end = 36)
df_viz['Exponential_Smoothing0.1'] = model2.predict(start = 0, end = 36)
df_viz['AutoReg'] = res.predict(start = 0, end = 36)

In [140]:
fig = px.area(df_viz.loc[ : '2012'], y = 'Bookings',
        title='Bookings bis 2013 und Trend/Vorhersage bis Ende 2013')

fig.update_traces(line = {'color' : 'lightGreen', 'width' : 0.3})

fig.add_scatter(x = df_viz.reset_index()['datetime'], y = df_viz.reset_index()['Prediction'], line = {'dash' : 'dot'}, name='Trend')
fig.add_scatter(x = df_viz.reset_index()['datetime'], y = df_viz.reset_index()['AutoReg'], line = {'color' : 'purple'}, name='Prognose' )
fig.add_scatter(x = df_viz.reset_index()['datetime'], y = df_viz.reset_index()['Exponential_Smoothing0.9'], line = {'color' : 'blue'}, name='Exponentielle_Glättung_0.9')
fig.add_scatter(x = df_viz.reset_index()['datetime'], y = df_viz.reset_index()['Exponential_Smoothing0.1'], line = {'color' : '#2E8B57'}, name='Exponentielle_Glättung_0.1')


fig.show()

In [110]:
print(f"Max von 2013 : {df_viz.loc['2013' : ]['AutoReg'].idxmax()} mit {df_viz.loc['2013' : ]['AutoReg'].max()} Bookings")
print(f"Min von 2013 : {df_viz.loc['2013' : ]['AutoReg'].idxmin()} mit {df_viz.loc['2013' : ]['AutoReg'].min()} Bookings")
print(f"Durchschnitt von 2013 : Average = {df_viz.loc['2013' : ]['AutoReg'].mean()} Bookings")
print(f"Summe von 2013 : Summe = {df_viz.loc['2013' : ]['AutoReg'].sum()} Bookings")
print(f"Die Prozentuale Veränderung gegenüber 2012 : {round((df_viz.loc['2013' : ]['AutoReg'].sum() / df_viz.loc['2012']['Bookings'].sum() -1 ) * 100,2)}%")

Max von 2013 : 2013-06-30 00:00:00 mit 276340.6249999997 Bookings
Min von 2013 : 2013-01-31 00:00:00 mit 167209.62499999983 Bookings
Durchschnitt von 2013 : Average = 239866.833333333 Bookings
Summe von 2013 : Summe = 2878401.999999996 Bookings
Die Prozentuale Veränderung gegenüber 2012 : 39.85%
